In [46]:
import os
import sys
import pandas as pd
filename = os.path.abspath('')+'/src'
sys.path.insert(0,filename)
import GPy
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from numpy.linalg import inv
from configurations import load_design, transform_design
from bayes_mcmc import *
from emulator import *
from calculations_load import trimmed_model_data
from bins_and_cuts import *
import matplotlib.patches as mpatches
from bayes_exp import Y_exp_data
from sklearn.preprocessing import StandardScaler

import pymc3 as pm

### Following are the map parameters found by 

In [47]:
MAP_params = {'Pb-Pb-2760': [14.128, 0.089, 1.054, 1.064, 4.227, 1.507, 0.113, 0.223, -1.585, 0.32, 0.056, 0.11, 0.16, 0.093, -0.084, 4.666, 0.136],
              'Au-Au-200' : [5.821, 0.089, 1.054, 1.064, 4.227, 1.507, 0.113, 0.223, -1.585, 0.32, 0.056, 0.11, 0.16, 0.093, -0.084, 4.666, 0.136]
                }

In [48]:
map_values=np.array(MAP_params["Pb-Pb-2760"])

In [49]:
MAP=transform_design(map_values.reshape(1,-1))

In [50]:
em=Trained_Emulators['Pb-Pb-2760']

In [51]:
em.gps[2].alpha

0.1

In [216]:
# a function that takes Scikit lern emulator as input and return the corresponding GPy emulator.
def convertToGPy(Scikit_emu):
    """Takes a scikitlearn gaussian emulator as input and return corresponding GPy gaussian emulator"""
    dic=Scikit_emu.kernel_.get_params()
    dim=len(dic["k1__k2"].get_params()["length_scale"])
    kernel1= GPy.kern.RBF(input_dim=dim, variance=dic['k1__k1__constant_value'], lengthscale=dic["k1__k2"].get_params()["length_scale"],ARD=True)
    kernal2=GPy.kern.White(input_dim=dim,variance=Scikit_emu.alpha)
    kernel=kernel1+kernal2
    m = GPy.models.GPRegression(Scikit_emu.X_train_,Scikit_emu.y_train_.reshape(-1,1),kernel,noise_var=dic["k2__noise_level"])
    #m.constrain_positive('') 
    #m.optimize_restarts(num_restarts = 10)
    return m

In [217]:
#make an dictionary of GPy emulators corresponding to each of the 10 PCA components of the observables
GPy_emulators={}
for i in range(0,10):
    EM=em.gps[i]
    #zz=Z.T[i].reshape(-1,1)
    #print(zz.reshape(-1,))
    #print(em.gps[-i].y_train_)
    #print(zz-em.gps[i].y_train_)
    GPy_emulators[i]=convertToGPy(Scikit_emu=EM)

In [218]:
GPy_emulators[5].predict(Xnew=MAP)

(array([[-0.70035257]]), array([[0.11507379]]))

In [219]:
em.gps[5].predict(MAP,return_cov=True)

(array([-0.70035257]), array([[0.01507379]]))

In [210]:
data=[]
for i in range(0,10):
    xp,yp=GPy_emulators[i].predict(Xnew=MAP)
    xs,ys=em.gps[i].predict(MAP,return_cov=True)
    data.append([i,xp.flatten(),xs.flatten(),xp.flatten()-xs.flatten(),yp.flatten(),ys.flatten(),yp.flatten()-ys.flatten()])

### prediction of the 10 principle components for the MAP parameter values found

In [211]:
pd.DataFrame(data, columns=["PCA component","Gpy mean","GPs mean","mean difference","Gpy Cov","GPs Cov","Cov difference"]) 

,PCA component,Gpy mean,GPs mean,mean difference,Gpy Cov,GPs Cov,Cov difference
0,0,[-0.02842021624609714],[-0.028420213041446374],[-3.204650766974737e-09],[0.11293328160662633],[0.012933281405764774],[0.10000000020086156]
1,1,[-0.1080975190947413],[-0.10809752009778606],[1.003044758363103e-09],[0.11325435582175772],[0.013254355592572153],[0.10000000022918556]
2,2,[-0.2639963713569786],[-0.2639963724026524],[1.0456737697950302e-09],[0.11455936737562702],[0.014559367085075436],[0.10000000029055159]
3,3,[-0.33658035230470773],[-0.3365803528460063],[5.412985615294019e-10],[0.11427806709265767],[0.014278066821100666],[0.100000000271557]
4,4,[-0.09244021557669235],[-0.0924402158943689],[3.176765517309832e-10],[0.11442255270189464],[0.014422552414210088],[0.10000000028768455]
5,5,[-0.7003525691208594],[-0.7003525691555055],[3.4646063795662485e-11],[0.11507379243633623],[0.015073792113875939],[0.10000000032246029]
6,6,[0.2808339815640668],[0.280833980442039],[1.122027804001391e-09],[0.11706744367912628],[0.01706744328029508],[0.1000000003988312]
7,7,[0.1809560011130089],[0.1809560034658686],[-2.3528596848620964e-09],[0.1202581006387721],[0.020258100100776666],[0.10000000053799543]
8,8,[0.6891829727880996],[0.6891829721262539],[6.618456893647817e-10],[0.1976432513454313],[0.09764325076695979],[0.1000000005784715]
9,9,[-0.4482157121386665],[-0.4482157184165376],[6.277871111848299e-09],[0.14924919968997355],[0.04924919835091579],[0.10000000133905776]


When I convert the GP from Scikit to Gpy there is always a difference of 0.1 in the variance. This is because to reproduce the same mean that I get from scikit learn GP I had to add a constant white noise of variance alpha to the kernal in Gpy. Otherwise the predicted means would not be the same. The mean would be different by a factor of +/- 10^(-2) if the value of alpha is not included as a constant kernal in Gpy emulator. 

I think this is due to the alpha paramter we use in the Scikit learn emulator training. For the trained emulators considered above alpha has been set to 0.1. Although this alpha value is added to the diagonal of the kernal while training it does not apear as part of the covariance matrix which is returend. But this alpha value is again used when the predictions are made from the trained gaussian emulator.

alpha is there to ensure that the kernel matrix is positive definite, so that it can be inverted without numerical error. It should always be set to a very small positive number so it won't affect the results. But this was not clear in there earlier documentaion and now scikit learn has fixed this bug and updated there documentation. Earlier they said alpha is a parameter resembling input noise variance and it is similar to adding a white noise kernal to the covariance matrix. Now they have changed this documentation and it has no resemblance to a white noise. 
The following is the link to this bug fix.
https://github.com/scikit-learn/scikit-learn/pull/15990

In [143]:
em.gps[0].X_train_.shape

(485, 29)

In [285]:
# a function that takes Scikit learn emulator as input and return the corresponding Pymc3 emulator.
def convertTopymc3(Scikit_emu,i,k):
    """Takes a scikitlearn gaussian emulator as input and return corresponding GPy gaussian emulator"""
    with pm.Model() as k:
            dic=em.gps[i].kernel_.get_params()
            Sigma2=float(dic['k1__k1__constant_value'])
            dim=len(dic["k1__k2"].get_params()["length_scale"])
            ccov=pm.gp.cov.ExpQuad(input_dim=dim, ls=dic["k1__k2"].get_params()["length_scale"])
            f_cov = Sigma2 * ccov
            cov_func_wnn=pm.gp.cov.WhiteNoise(np.sqrt(em.gps[i].alpha+dic["k2__noise_level"]))
           # cov_func_wn=np.sqrt(em.gps[0].alpha+dic["k2__noise_level"])
            m = pm.gp.Marginal(cov_func=f_cov+cov_func_wnn)
            yt= m.marginal_likelihood("yt",X=em.gps[i].X_train_,y=em.gps[i].y_train_.reshape(-1,1),noise=0)
            mu, var = m.predict(Xnew=MAP,diag=True, pred_noise=False)
            print(mu[0][0],var[0])
            return mu[0][0],var[0]

In [268]:
with pm.Model() as model:
        dic=em.gps[0].kernel_.get_params()
        Sigma2=float(dic['k1__k1__constant_value'])
        dim=len(dic["k1__k2"].get_params()["length_scale"])
        ccov=pm.gp.cov.ExpQuad(input_dim=dim, ls=dic["k1__k2"].get_params()["length_scale"])
        f_cov = Sigma2 * ccov
        cov_func_wnn=pm.gp.cov.WhiteNoise(np.sqrt(em.gps[0].alpha+dic["k2__noise_level"]))
       # cov_func_wn=np.sqrt(em.gps[0].alpha+dic["k2__noise_level"])
        m = pm.gp.Marginal(cov_func=f_cov+cov_func_wnn)
        yt= m.marginal_likelihood("yt",X=em.gps[0].X_train_,y=em.gps[0].y_train_.reshape(-1,1),noise=0)
        mu, var = m.predict(Xnew=MAP,diag=True, pred_noise=False)
        print(mu[0][0],var[0])

-0.028420533467709338 0.11293330149269565


In [287]:
#make an dictionary of GPy emulators corresponding to each of the 10 PCA components of the observables
pymc3_emulators_predition={}
for i in range(0,10):
    EM=em.gps[i]
    pymc3_emulators_predition[i]=convertTopymc3(Scikit_emu=EM,i=i,k=i)

-0.028420533467709338 0.11293330149269565
-0.10809741977194727 0.11325437851029108
-0.26399626783542635 0.11455939614026178
-0.3365802987095955 0.11427809397632593
-0.09244018413987454 0.11442258118283277
-0.700352565654405 0.11507382435891778
0.28083409268821924 0.11706748316297588
0.18095576813667838 0.12025815389845551
0.6891830383046265 0.1976433086122169
-0.4482150906037686 0.14924933225854264


In [289]:
data=[]
for i in range(0,10):
    xp,yp=GPy_emulators[i].predict(Xnew=MAP)
    xs,ys=em.gps[i].predict(MAP,return_cov=True)
    xpm,ypm=pymc3_emulators_predition[i]
    data.append([i,xp.flatten(),xs.flatten(),xpm,yp.flatten(),ys.flatten(),ypm])

In [290]:
pd.DataFrame(data, columns=["PCA component","Gpy mean","GPs mean","Pymc3 GP mean","Gpy Cov","GPs Cov","Pymc3 GP Cov"]) 

,PCA component,Gpy mean,GPs mean,Pymc3 GP mean,Gpy Cov,GPs Cov,Pymc3 GP Cov
0,0,[-0.02842021624609714],[-0.028420213041446374],-0.028421,[0.11293328160662619],[0.012933281405764774],0.112933
1,1,[-0.1080975190947413],[-0.10809752009778606],-0.108097,[0.11325435582176113],[0.013254355592572153],0.113254
2,2,[-0.2639963713569786],[-0.2639963724026524],-0.263996,[0.11455936737562689],[0.014559367085075436],0.114559
3,3,[-0.33658035230470773],[-0.3365803528460063],-0.336580,[0.11427806709265753],[0.014278066821100666],0.114278
4,4,[-0.09244021557669235],[-0.0924402158943689],-0.092440,[0.1144225527018945],[0.014422552414210088],0.114423
5,5,[-0.7003525691208594],[-0.7003525691555055],-0.700353,[0.11507379243633609],[0.015073792113875939],0.115074
6,6,[0.2808339815640668],[0.280833980442039],0.280834,[0.11706744367912614],[0.01706744328029508],0.117067
7,7,[0.1809560011130089],[0.1809560034658686],0.180956,[0.12025810063877196],[0.020258100100776666],0.120258
8,8,[0.6891829727880996],[0.6891829721262539],0.689183,[0.1976432513454312],[0.09764325076695979],0.197643
9,9,[-0.4482157121386665],[-0.4482157184165376],-0.448215,[0.1492491996899734],[0.04924919835091579],0.149249


#### Here also to get the same mean value that is predicted by the scikit learn GP I had to add diagonal white noise with variance of the alpha value to the kernal. Other wise I do not get the same mean value. So Although the alpha value does not apear in the final prediction variance it is in the kernal and it affects the results. It is not going to be optimized during the training. 